<img src="datamecum_logo.png" align="right" style="float" width="400">
<font color="#CA3532"><h1 align="left">Programa técnico intensivo en data science. Datamecum.</h1></font>
<font color="#6E6E6E"><h2 align="left">Decision Trees</h2></font> 
<font color="#6E6E6E"><h2 align="left">Construcción del árbol</h2></font> 

### Calculo de la entropía

Podemos generalizarla para una variable aleatoria discreta ``X`` de ``n`` resultados como:

$$
    H(X) = - \sum_{i=1}^n p(x_{i}) * log_{2}(p(x_{i}))
$$

In [36]:
from __future__ import division
from math import log

In [37]:
def H(values):
    valor = 0

    for value in values:
        if (value > 0):
            valor = valor + value * log(value, 2)
    
    return - valor

## Arboles de Decisión: ID3

- Algoritmo de Tipo Greedy
    - En cada paso realiza el mejor split posible en dos con un cierto criterio:
        - Seleccionar el atributo que nos da mayor **Ganancia de Información**.
    - Proceso se repite recursivamente hasta construir el árbol final.

La **Ganancia de informacion** se ira calculando en cada paso, calculando la diferencia entre la entropia antes de realizar un split y despues de realizar el mismo por un cierto feature. Esta se puede interpretar como que al realizar un determinado split, se reduce la incertidumbre de la prediccion en ese sub-arbol dado un cierto valor.

El hecho de que el algoritmo sea greedy se manifiesta en que nos quedaremos con aquel feature que nos da **mayor ganancia de informacion**.

Veremos un ejemplo para poder comprender el concepto

### Set de datos

| candidato | presencia | estudios | experiencia | contratado  |
|-----------|-----------|----------|-------------|-------------|
| 1			| buena		| univ	   | alta   	 | *si*		   |
| 2			| mala		| univ	   | media  	 | **no**	   |
| 3			| buena		| sec	   | alta   	 | *si*		   |
| 4			| mala		| univ	   | baja   	 | **no**	   |
| 5			| buena		| sec	   | media	 	 | *si*		   |
| 6			| buena		| univ	   | media  	 | *si*		   |
| 7			| reg		| pri	   | baja   	 | **no**	   |
| 8			| reg		| univ	   | media  	 | *si*		   |

### ¿Qué queremos predecir?

Queremos predecir si la persona **será o no sera contratada** y siempre sera la base que utilizaremos para calcular la entropia.

### Features a considerar

- presencia
- estudios
- experiencia

### Primer Nivel

Calculamos la entropia del set de datos donde tenemos:

- 8 candidatos
    - **5 FUERON contratados**
    - **3 NO FUERON contratados**
    
Calculamos entonces la entropía del set de datos (antes de realizar un split):

In [38]:
H([5/8, 3/8])

0.9544340029249649

In [39]:
h_before = H([5/8, 3/8])

Si consideramos como candidato para el split nuestro primer atributo **presencia** podemos ver que cuenta con los siguientes valores:

- buena
- mala
- regular

y a su vez podemos considerar parcialmente cada uno de los casos que se corresponden con lo que queremos *predecir*.

|presencia | contratado  | no contratado |
|----------|-------------|---------------|
| buena	   | 4           |0|   
| mala	   | 0           |2| 
| reg	   | 1           |1|

Para poder calcular la **ganancia de informacion** tenemos que obtener la **entropia para el feature presencia**. Para ello calculamos la entropia para cada valor posible del feature.

In [40]:
#H(presencia=buena)
H([4/4,0])

-0.0

In [41]:
#H(presencia=mala)
H([0,2/2])

-0.0

In [42]:
#H(presencia=reg)
H([1/2,1/2])

1.0

y por ultimo podemos expresar la **entropia del feature presencia** como **el promedio ponderado de la entropia de cada valor posible por la probabilidad de que el atributo tome dicho valor**.

In [43]:
# H(presencia)
4/8 * H([4/4,0]) + 2/8 * H([0,2/2]) + 2/8 * H([1/2,1/2])

0.25

In [46]:
h_after_presence = 4/8 * H([4/4,0]) + 2/8 * H([0,2/2]) + 2/8 * H([1/2,1/2])

Por ultimo el calculo de la **Ganancia de Informacion** lo realizamos como la diferencia entre la entropia del set de datos (al inicio) y la entropia del atributo

In [47]:
#IG(presencia)
h_before - h_after_presence

0.7044340029249649

De la misma forma podemos realizar el mismo calculo para los otros atributos. Por ejemplo la ganancia de la información de estudios.

In [16]:
h_before - (1/8 * H([0,1/1]) + 2/8 * H([2/2,0]) + 5/8 * H([3/5,2/5]))

0.34758988139079705

Tambien podemos repetir el proceso para la ganancia de la información de la experiencia.

In [17]:
h_before - (2/8 * H([2/2,0]) + 4/8 * H([1/4, 3/4]) + 2/8 * H([0, 2/2]))

0.5487949406953985

El **feature/atributo con mayor ganancia de informacion** es *presencia* por lo que sera nuestra raiz del arbol de decision.

Podemos ver la siguiente representacion del mismo. Es importante notar que nuestro arbol de decision ya clasifica correctamente los casos de presencia buena y mala (no quedando casos sin clasificar).

```
         presencia
         [5si,3no]              

    /        |       \                 
  =buena   =mala     =reg
 -------  --------  -------
[4si,0no] [0si,2no] [1si,1no]
```

### Segundo Nivel

Por ultimo nos quedan los casos de regular, en los cuales tenemos un contratado y un no contratado y dos features a evaluar: estudios y experiencia. 

Para poder hacerlo debemos aplicar nuevamente el algoritmo recursivamente considerando el nuevo set de datos con los elementos que quedan en ese sub-set de datos del nodo (los dos dentro cuya presencia fue regular).

Tenemos que calcular la la Ganancia de Informacion, pero en este caso podemos hacer esto sin cuentas porque observamos que en los dos casos en los cuales la presencia es regular podemos usar tanto los estudios como la experiencia para hacer el split, por lo que un posible ejemplo podria ser el siguiente:

```
         presencia
         [5si,3no]               
                                 
    /        |       \                 
  =buena   =mala     estudios
 -------  --------    -------
[4si,0no] [0si,2no]  [1si,1no]
                    
                     /        \
                   =pri      =univ
                 [0si,1no]  [1si,0no]
```


Ejercicios: Comprobar si el la seleccion de la variable en el segundo nivel está bien realizada.